In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Thu Jul 30 11:36:00 PDT 2020
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.990
BogoMIPS:              5851.88
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables

In [19]:
# Set data directories
%env data_dir=/home/sam/data/P_generosa
%env fasta=/home/sam/data/P_generosa/Panopea-generosa-vv0.74.a4.5d9637f372b5d-publish.genes.fna
%env out_dir=/home/sam/analyses/20200730_pgen_primer_design

# Needed for primer3-2.4.0
%env thermo_params_dir=/home/sam/programs/primer3-2.4.0/src/primer3_config/

# Programs
%env primer3=/home/sam/programs/primer3-2.4.0/src/primer3_core
%env primersearch=/home/sam/programs/EMBOSS-6.6.0/emboss/primersearch

env: data_dir=/home/sam/data/P_generosa
env: fasta=/home/sam/data/P_generosa/Panopea-generosa-vv0.74.a4.5d9637f372b5d-publish.genes.fna
env: out_dir=/home/sam/analyses/20200730_pgen_primer_design
env: thermo_params_dir=/home/sam/programs/primer3-2.4.0/src/primer3_config/
env: primer3=/home/sam/programs/primer3-2.4.0/src/primer3_core
env: primersearch=/home/sam/programs/EMBOSS-6.6.0/emboss/primersearch


#### Make directories

In [3]:
%%bash
mkdir --parents "${out_dir}"

### Download P.generosa genes FastA file from OSF repo (https://osf.io/ct623/)

In [6]:
%%bash

cd ${data_dir}

wget --quiet "https://files.osf.io/v1/resources/yem8n/providers/osfstorage/5db35d9abc32f4000e0b70c2?action=download&direct&version=1" \
--output-document ${fasta}

ls -lh

total 376M
-rw-rw-r-- 1 sam sam 375M Jul 30 11:53 Panopea-generosa-vv0.74.a4.5d9637f372b5d-publish.genes.fna
-rw-rw-r-- 1 sam sam 1.6M Jul 29 11:48 Panopea-generosa-vv0.74.a4.5d9637f372b5d-publish.genes.fna.fai


### Extract target sequences from FastA

In [20]:
%%bash

timestamp=$(date +"%Y%m%d")

cd "${out_dir}"

# Associative array to associate gene names with sequence ids
# Requires >= Bash 4.0
declare -A seqid_array

# Populate associative array [gene_abbreviation]=seqid
seqid_array=(
[TIF3s6b]=PGEN_.00g000750-vv0.74.a \
[TIF3s12]=PGEN_.00g025890-vv0.74.a \
[APLP]=PGEN_.00g070040-vv0.74.a \
[TIF3s7]=PGEN_.00g079690-vv0.74.a \
[TIF3s5]=PGEN_.00g082590-vv0.74.a \
[NFIP1]=PGEN_.00g088260-vv0.74.a \
[GSK3B]=PGEN_.00g114060-vv0.74.a \
[TIF3s8-1]=PGEN_.00g132030-vv0.74.a \
[TIF3s8-2]=PGEN_.00g132040-vv0.74.a \
[FEN1]=PGEN_.00g188130-vv0.74.a \
[ECHD3]=PGEN_.00g194630-vv0.74.a \
[GLYG]=PGEN_.00g224740-vv0.74.a \
[TIF3s10]=PGEN_.00g245080-vv0.74.a \
[SPTN1]=PGEN_.00g280110-vv0.74.a \
[RPL5]=PGEN_.00g287540-vv0.74.a \
[TIF3s4a]=PGEN_.00g288180-vv0.74.a \
[NSF]=PGEN_.00g338640-vv0.74.a
)

# Individual FastAs array
fasta_array=()

# Extract sequences to individual FastA files
for gene_name in "${!seqid_array[@]}"
do
    # Set output file names
    out_file="${seqid_array[$gene_name]}"_"${gene_name}".fna

    # Run faidx
    faidx  "${fasta}" "${seqid_array[$gene_name]}" \
    --out "${out_file}"
    
    # Add FastA to array
    fasta_array+=(${out_file})
    
    ## Check output
    # Count number of entries in output FastA (should be = 1)
    # -H displays filename - is compatible with OSX
    grep --count -H ">" "${out_file}"
    
    # Check each FastA header
    echo "${out_file}: $(head -n1 "${out_file}")"
    echo ""
    
done

# Run Primer3
for fna in "${fasta_array[@]}"
do

    # Store sequence only from desired FastA.
    # Print all lines after the first line and then delete newlines
    # because sequence needs to be on single line for Primer3 params file
    sequence=$(tail -n +2 "${fna}" | tr -d '\n')
    
    # Remove suffix from FastA file to use as sequence ID
    seq_id=${fna%.*}
    
    # Variables for output filenaming
    params_out="${timestamp}_${seq_id}_primer3_params.txt"
    primer3_def_out="${timestamp}_${seq_id}_primers_default_format.txt"
    emboss_primers="${timestamp}_${seq_id}_emboss_primers.txt"
    primersearch_out="${timestamp}_${seq_id}_primersearch.txt"
    
    # Use heredoc to create Primer3 parameters file
    
    ##Values after the "=" on each line can be changed to whatever values the user decides. 
    ##The ${sequence} must be a nucletoide sequence on a single line, with no line breaks.

    ## The code below uses a ```heredoc``` to write this information to a file.
    ## Everything between the following two lines gets printed (via ```cat```) as shown and then
    ## redirected to the indicated file "${params_out}".
    
    ## BTW, heredoc cannot be indented (well, it can, but requires some formatting that I couldn't get to work)
cat << EOF > "${params_out}"
SEQUENCE_ID=${seq_id}
SEQUENCE_TEMPLATE=${sequence}
PRIMER_TASK=generic
PRIMER_PICK_LEFT_PRIMER=3
PRIMER_PICK_RIGHT_PRIMER=3
PRIMER_OPT_SIZE=18
PRIMER_MIN_SIZE=15
PRIMER_MAX_SIZE=21
PRIMER_MAX_NS_ACCEPTED=1
PRIMER_PRODUCT_SIZE_RANGE=75-150
P3_FILE_FLAG=1
PRIMER_EXPLAIN_FLAG=1
PRIMER_THERMODYNAMIC_PARAMETERS_PATH=${thermo_params_dir}
=
EOF
    
    # Run Primer3 with default output format
    ${primer3} \
    --output="${out_dir}/${primer3_def_out}" \
    "${params_out}"
    
    # Create tab-delimited primer file for primersearch
    sequence_id=$(grep "SEQUENCE_ID=" "${primer3_def_out}" | sed 's/SEQUENCE_ID=//')
    left_primer=$(grep "PRIMER_LEFT_0_SEQUENCE=" "${primer3_def_out}" | sed 's/PRIMER_LEFT_0_SEQUENCE=//')
    right_primer=$(grep "PRIMER_RIGHT_0_SEQUENCE=" "${primer3_def_out}" | sed 's/PRIMER_RIGHT_0_SEQUENCE=//')
    
    printf "%s\t" "${sequence_id}" "${left_primer}" "${right_primer}" > "${emboss_primers}"
    
    # Add required newline to end of file
    printf "\n" >> "${emboss_primers}"
    
    # Run EMBOSS primersearch
    ${primersearch} \
    -seqall "${fasta}" \
    -infile "${emboss_primers}" \
    -mismatchpercent 20 \
    -outfile "${primersearch_out}" \
    -auto
    

done

PGEN_.00g194630-vv0.74.a_ECHD3.fna:1
PGEN_.00g194630-vv0.74.a_ECHD3.fna: >PGEN_.00g194630-vv0.74.a

PGEN_.00g288180-vv0.74.a_TIF3s4a.fna:1
PGEN_.00g288180-vv0.74.a_TIF3s4a.fna: >PGEN_.00g288180-vv0.74.a

PGEN_.00g132040-vv0.74.a_TIF3s8-2.fna:1
PGEN_.00g132040-vv0.74.a_TIF3s8-2.fna: >PGEN_.00g132040-vv0.74.a

PGEN_.00g132030-vv0.74.a_TIF3s8-1.fna:1
PGEN_.00g132030-vv0.74.a_TIF3s8-1.fna: >PGEN_.00g132030-vv0.74.a

PGEN_.00g338640-vv0.74.a_NSF.fna:1
PGEN_.00g338640-vv0.74.a_NSF.fna: >PGEN_.00g338640-vv0.74.a

PGEN_.00g245080-vv0.74.a_TIF3s10.fna:1
PGEN_.00g245080-vv0.74.a_TIF3s10.fna: >PGEN_.00g245080-vv0.74.a

PGEN_.00g025890-vv0.74.a_TIF3s12.fna:1
PGEN_.00g025890-vv0.74.a_TIF3s12.fna: >PGEN_.00g025890-vv0.74.a

PGEN_.00g114060-vv0.74.a_GSK3B.fna:1
PGEN_.00g114060-vv0.74.a_GSK3B.fna: >PGEN_.00g114060-vv0.74.a

PGEN_.00g287540-vv0.74.a_RPL5.fna:1
PGEN_.00g287540-vv0.74.a_RPL5.fna: >PGEN_.00g287540-vv0.74.a

PGEN_.00g188130-vv0.74.a_FEN1.fna:1
PGEN_.00g188130-vv0.74.a_FEN1.fna: >PGEN_.00g1

### List output files

In [21]:
%%bash

cd "${out_dir}"

ls -ltrh

total 28M
-rw-rw-r-- 1 sam sam  15K Jul 30 21:13 PGEN_.00g194630-vv0.74.a_ECHD3.fna
-rw-rw-r-- 1 sam sam  12K Jul 30 21:13 PGEN_.00g288180-vv0.74.a_TIF3s4a.fna
-rw-rw-r-- 1 sam sam 4.9K Jul 30 21:13 PGEN_.00g132040-vv0.74.a_TIF3s8-2.fna
-rw-rw-r-- 1 sam sam  17K Jul 30 21:13 PGEN_.00g132030-vv0.74.a_TIF3s8-1.fna
-rw-rw-r-- 1 sam sam  21K Jul 30 21:13 PGEN_.00g338640-vv0.74.a_NSF.fna
-rw-rw-r-- 1 sam sam  27K Jul 30 21:13 PGEN_.00g245080-vv0.74.a_TIF3s10.fna
-rw-rw-r-- 1 sam sam 9.8K Jul 30 21:13 PGEN_.00g025890-vv0.74.a_TIF3s12.fna
-rw-rw-r-- 1 sam sam  27K Jul 30 21:13 PGEN_.00g114060-vv0.74.a_GSK3B.fna
-rw-rw-r-- 1 sam sam  14K Jul 30 21:13 PGEN_.00g287540-vv0.74.a_RPL5.fna
-rw-rw-r-- 1 sam sam  17K Jul 30 21:13 PGEN_.00g188130-vv0.74.a_FEN1.fna
-rw-rw-r-- 1 sam sam  48K Jul 30 21:13 PGEN_.00g224740-vv0.74.a_GLYG.fna
-rw-rw-r-- 1 sam sam 8.4K Jul 30 21:13 PGEN_.00g088260-vv0.74.a_NFIP1.fna
-rw-rw-r-- 1 sam sam  27K Jul 30 21:13 PGEN_.00g082590-vv0.74.a_TIF3s5.fna
-rw-rw-r-- 1 sam sam

### Count number of primer matches identified for each primer set

Reminder: Each set should match at least once - to its own gene sequence from which it was derived.

Reminder: These primers were designed against gene sequences, _not_ coding sequences.

In [22]:
%%bash

cd "${out_dir}"

grep -c "Amplimer*" *primersearch.txt

20200730_PGEN_.00g000750-vv0.74.a_TIF3s6b_primersearch.txt:15512
20200730_PGEN_.00g025890-vv0.74.a_TIF3s12_primersearch.txt:2
20200730_PGEN_.00g070040-vv0.74.a_APLP_primersearch.txt:2
20200730_PGEN_.00g079690-vv0.74.a_TIF3s7_primersearch.txt:14
20200730_PGEN_.00g082590-vv0.74.a_TIF3s5_primersearch.txt:742
20200730_PGEN_.00g088260-vv0.74.a_NFIP1_primersearch.txt:36
20200730_PGEN_.00g114060-vv0.74.a_GSK3B_primersearch.txt:8596
20200730_PGEN_.00g132030-vv0.74.a_TIF3s8-1_primersearch.txt:10
20200730_PGEN_.00g132040-vv0.74.a_TIF3s8-2_primersearch.txt:7800
20200730_PGEN_.00g188130-vv0.74.a_FEN1_primersearch.txt:2
20200730_PGEN_.00g194630-vv0.74.a_ECHD3_primersearch.txt:2
20200730_PGEN_.00g224740-vv0.74.a_GLYG_primersearch.txt:46
20200730_PGEN_.00g245080-vv0.74.a_TIF3s10_primersearch.txt:8
20200730_PGEN_.00g280110-vv0.74.a_SPTN1_primersearch.txt:496
20200730_PGEN_.00g287540-vv0.74.a_RPL5_primersearch.txt:2570
20200730_PGEN_.00g288180-vv0.74.a_TIF3s4a_primersearch.txt:4
20200730_PGEN_.00g33864

### Print program options, for reference

#### Pyfaidx command line program

In [24]:
%%bash
faidx -h

usage: faidx [-h] [-b BED] [-o OUT]
             [-i {bed,chromsizes,nucleotide,transposed}] [-c] [-r] [-y]
             [-a SIZE_RANGE] [-n | -f] [-t] [-x] [-l] [-s DEFAULT_SEQ]
             [-d DELIMITER] [-e HEADER_FUNCTION]
             [-u {stop,first,last,longest,shortest}] [-g REGEX] [-v] [-m | -M]
             [--no-output] [--no-rebuild] [--version]
             fasta [regions [regions ...]]

Fetch sequences from FASTA. If no regions are specified, all entries in the
input file are returned. Input FASTA file must be consistently line-wrapped,
and line wrapping of output is based on input line lengths.

positional arguments:
  fasta                 FASTA file
  regions               space separated regions of sequence to fetch e.g.
                        chr1:1-1000

optional arguments:
  -h, --help            show this help message and exit
  --no-rebuild          do not rebuild the .fai index even if it is out of
                        date. default: False
  --version      

#### Primer3

In [26]:
%%bash
${primer3} -help


Copyright (c) 1996-2017
Whitehead Institute for Biomedical Research, Steve Rozen
(http://purl.com/STEVEROZEN/), Andreas Untergasser and Helen Skaletsky
All rights reserved.

    This file is part of the primer3 suite and libraries.

    The primer3 suite and libraries are free software;
    you can redistribute them and/or modify them under the terms
    of the GNU General Public License as published by the Free
    Software Foundation; either version 2 of the License, or (at
    your option) any later version.

    This software is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this software (file gpl-2.0.txt in the source
    distribution); if not, write to the Free Software
    Foundation, Inc., 51 Franklin St, Fifth 

CalledProcessError: Command 'b'${primer3} -help\n'' returned non-zero exit status 255.

#### EMBOSS primersearch

In [27]:
%%bash
${primersearch} -help -verbose

Search DNA sequences for matches with primer pairs
Version: EMBOSS:6.6.0.0

   Standard (Mandatory) qualifiers:
  [-seqall]            seqall     Nucleotide sequence(s) filename and optional
                                  format, or reference (input USA)
  [-infile]            infile     Primer pairs file
  [-mismatchpercent]   integer    [0] Allowed percent mismatch (Any integer
                                  value)
  [-outfile]           outfile    [*.primersearch] Whitehead primer3_core
                                  program output file

   Additional (Optional) qualifiers: (none)
   Advanced (Unprompted) qualifiers: (none)
   Associated qualifiers:

   "-seqall" associated qualifiers
   -sbegin1            integer    Start of each sequence to be used
   -send1              integer    End of each sequence to be used
   -sreverse1          boolean    Reverse (if DNA)
   -sask1              boolean    Ask for begin/end/reverse
   -snucleotide1       boolean    Sequence is nuc